# __Code Challenge__

In [6]:
import pandas as pd
import numpy as np
from string import punctuation
from textdistance import levenshtein

### __Read Files__

In [7]:
example_abp_data = pd.read_csv('C://Users/Admin/Identeqchallenge/example_abp_data.csv')
example_abp_data.head()

SINGLE_LINE_ADDRESS POSTCODE    STREET_NAME
0  PACEC, SECOND FLOOR, 49-53 REGENT STREET, CAMB...  CB2 1AB  REGENT STREET
1  FIRST FLOOR, 47-53 REGENT STREET, CAMBRIDGE, C...  CB2 1AB  REGENT STREET
2  BUBBLE LTD, SUITE 1, RAVENSCROFT HOUSE, 59-61 ...  CB2 1AB  REGENT STREET
3  EDE & RAVENSCROFT, SUITE 2, RAVENSCROFT HOUSE,...  CB2 1AB  REGENT STREET
4  CAMBRIDGE NUTRACEUTICALS LTD, SUITE 3, RAVENSC...  CB2 1AB  REGENT STREET

In [8]:
df = pd.read_csv('C://Users/Admin/Identeqchallenge/example_input_data.csv')
df.head()

Address_Line_1         Address_Line_2  \
0                            UNIT 3  39-41 MOORBRIDGE ROAD   
1                        4 WORKSHOP             PERCY ROAD   
2                       SAXON HOUSE             HENSON WAY   
3                     1 QUARRY ROAD            NORTHAMPTON   
4  METER R/O UNIT 54 UNKNOWN SUPPLY            DALTON PARK   

                  Address_Line_3 Address_Line_4 Address_Line_5  Postcode  
0                     MAIDENHEAD      BERKSHIRE            NaN   SL6 8LT  
1                     DARLINGTON            NaN            NaN   DL3 0RX  
2  TELFORD WAY INDUSTRIAL ESTATE      KETTERING       NN16 8PX  NN16 8PX  
3                        NN5 6NT            NaN            NaN   NN5 6NT  
4                         MURTON         SEAHAM            NaN   SR7 9HU

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Address_Line_1  100 non-null    object
 1   Address_Line_2  90 non-null     object
 2   Address_Line_3  83 non-null     object
 3   Address_Line_4  45 non-null     object
 4   Address_Line_5  14 non-null     object
 5   Postcode        100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [10]:
# Check memory to optimise for large datasets
df.memory_usage(deep=True)
# Can change data type from object to category when att cardinality is low

Index              128
Address_Line_1    7290
Address_Line_2    6584
Address_Line_3    6179
Address_Line_4    4739
Address_Line_5    3654
Postcode          6452
dtype: int64

## ___Preprocessing___

In [11]:
df = df.applymap(lambda s: s.upper() if type(s) == str else s) # make values upper case
df

Address_Line_1         Address_Line_2  \
0                              UNIT 3  39-41 MOORBRIDGE ROAD   
1                          4 WORKSHOP             PERCY ROAD   
2                         SAXON HOUSE             HENSON WAY   
3                       1 QUARRY ROAD            NORTHAMPTON   
4    METER R/O UNIT 54 UNKNOWN SUPPLY            DALTON PARK   
..                                ...                    ...   
95                    LAND OFF PLOT 8       DRAIN BANK NORTH   
96                        UNITS 8A/8B                 UNIT 2   
97                           BINSTORE             BRUFF ROAD   
98                       THE PHARMACY       RAVENSWORTH ROAD   
99  OFFICE 13 GND FLR, GROUNDSTAR HSE        FREIGHT VILLAGE   

                   Address_Line_3       Address_Line_4 Address_Line_5  \
0                      MAIDENHEAD            BERKSHIRE            NaN   
1                      DARLINGTON                  NaN            NaN   
2   TELFORD WAY INDUSTRIAL ESTATE            KETTERING       NN16 8PX   
3                         NN5 6NT                  NaN            NaN   
4                          MURTON               SEAHAM            NaN   
..                            ...                  ...            ...   
95                         COWBIT             SPALDING            NaN   
96          RIVERSIDE RETAIL PARK              NORWICH            NaN   
97                        IPSWICH              IP2 8EF            NaN   
98                      GATESHEAD                  NaN            NaN   
99                    WOOLSINGTON  NEWCASTLE UPON TYNE            NaN   

    Postcode  
0    SL6 8LT  
1    DL3 0RX  
2   NN16 8PX  
3    NN5 6NT  
4    SR7 9HU  
..       ...  
95  PE12 6FR  
96   NR1 1WR  
97   IP2 8EF  
98  NE11 9AZ  
99  NE13 8BH  

[100 rows x 6 columns]

In [12]:
df = df.apply(lambda x: x.str.strip()) # strip string values

In [13]:
df = df.apply(lambda x: x.str.strip(punctuation)) # strip punctuation

In [14]:
df.fillna('', inplace=True)
df.head()

Address_Line_1         Address_Line_2  \
0                            UNIT 3  39-41 MOORBRIDGE ROAD   
1                        4 WORKSHOP             PERCY ROAD   
2                       SAXON HOUSE             HENSON WAY   
3                     1 QUARRY ROAD            NORTHAMPTON   
4  METER R/O UNIT 54 UNKNOWN SUPPLY            DALTON PARK   

                  Address_Line_3 Address_Line_4 Address_Line_5  Postcode  
0                     MAIDENHEAD      BERKSHIRE                  SL6 8LT  
1                     DARLINGTON                                 DL3 0RX  
2  TELFORD WAY INDUSTRIAL ESTATE      KETTERING       NN16 8PX  NN16 8PX  
3                        NN5 6NT                                 NN5 6NT  
4                         MURTON         SEAHAM                  SR7 9HU

In [15]:
Postcode = df['Postcode']
df.pop('Postcode') # temporarily remove postcode to apply regex replace methods

df = df.replace(r'[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}', '', regex=True).astype('string') # remove redundant postcode in address line columns
df = df.replace(' RD', ' ROAD', regex=True).astype('string') # normalise street names

df['Single_Line_Address'] = df.apply(lambda x: ', '.join(x[x != '']), axis=1)

In [16]:
df.drop(['Address_Line_1', 'Address_Line_2', 'Address_Line_3', 'Address_Line_4', 'Address_Line_5'], axis=1, inplace=True) # remove extra columns
df.insert(1, 'Postcode', Postcode) # reinsert postcode

In [17]:
df.head()

Single_Line_Address  Postcode
0  UNIT 3, 39-41 MOORBRIDGE ROAD, MAIDENHEAD, BER...   SL6 8LT
1                 4 WORKSHOP, PERCY ROAD, DARLINGTON   DL3 0RX
2  SAXON HOUSE, HENSON WAY, TELFORD WAY INDUSTRIA...  NN16 8PX
3                         1 QUARRY ROAD, NORTHAMPTON   NN5 6NT
4  METER R/O UNIT 54 UNKNOWN SUPPLY, DALTON PARK,...   SR7 9HU

In [18]:
# Create dictionary with postcode as key and street names as values
from collections import defaultdict

post_street = defaultdict(list)

post_list = example_abp_data.POSTCODE.tolist()

street_list = example_abp_data.STREET_NAME.tolist()


for k, v in zip(post_list, street_list):
    if v not in post_street[k]:
        post_street[k].append(v)

print(post_street)

defaultdict(<class 'list'>, {'CB2 1AB': ['REGENT STREET'], 'CB22 4PS': ['MOORFIELD ROAD'], 'CM1 7FG': ['PUDDINGS WOOD DRIVE', 'WOODHOUSE LANE'], 'CM8 2AP': ['NEWLANDS PRECINCT', 'NEWLANDS DRIVE'], 'CM8 2DZ': ['COLLINGWOOD ROAD'], 'CO10 7UR': ['SCOSSELS'], 'CO15 4LP': ['TELFORD ROAD'], 'CO3 8PB': ['LONDON ROAD', 'CHURCH LANE'], 'CO6 1TU': ['CHURCH STREET'], 'DH4 5AL': ['SUNRISE LANE'], 'DL1 2PA': ['DODSWORTH STREET'], 'DL1 2PB': ['CLEVELAND TRADING ESTATE', 'FORGE WAY', 'PARK LANE', 'CLEVELAND STREET'], 'DL3 0RX': ['PERCY ROAD'], 'DL3 6LP': ['TEMPERANCE PLACE', 'FOUR RIGGS'], 'DN15 8RE': ['MERCIA WAY'], 'DN21 5QR': ['MIDDLE STREET'], 'DN9 3DW': ['CROFT ROAD', 'LOW DEEPS LANE', 'BRACKEN HILL LANE'], 'IG11 7BP': ['FRESH WHARF ROAD'], 'IG11 9RT': ['BLAKE AVENUE'], 'IP18 6DD': ['CAUTLEY ROAD'], 'IP2 8EF': ['CROFT STREET'], 'IP2 8HD': ['FELAW STREET'], 'IP28 6JH': ['THE STREET'], 'IP3 9BJ': ['FELIXSTOWE ROAD', 'HINES ROAD'], 'IP6 8NZ': ['MAITLAND ROAD', 'LION LANE'], 'IP8 3JD': ['OLD LONDON 

In [19]:
# Add expected street column
df['Expected_Street'] = df['Postcode'].map(post_street)
df.head()

Single_Line_Address  Postcode  \
0  UNIT 3, 39-41 MOORBRIDGE ROAD, MAIDENHEAD, BER...   SL6 8LT   
1                 4 WORKSHOP, PERCY ROAD, DARLINGTON   DL3 0RX   
2  SAXON HOUSE, HENSON WAY, TELFORD WAY INDUSTRIA...  NN16 8PX   
3                         1 QUARRY ROAD, NORTHAMPTON   NN5 6NT   
4  METER R/O UNIT 54 UNKNOWN SUPPLY, DALTON PARK,...   SR7 9HU   

                                     Expected_Street  
0                                                 []  
1                                       [PERCY ROAD]  
2  [HENSON WAY, VERNON COURT, RILEY ROAD, TELFORD...  
3                                      [QUARRY ROAD]  
4                                              [nan]

In [20]:
# Options to expand dataframe
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [21]:
df['Street_In_Postcode'] = df.apply(lambda x: 'Yes' if any(str(street) in x.Single_Line_Address for street in x.Expected_Street) else 'No', axis=1) # apply lambda function for matching

# ___Results___

In [22]:
df.loc[df['Street_In_Postcode'] == 'Yes']

Single_Line_Address  \
1                                                       4 WORKSHOP, PERCY ROAD, DARLINGTON   
2                        SAXON HOUSE, HENSON WAY, TELFORD WAY INDUSTRIAL ESTATE, KETTERING   
3                                                               1 QUARRY ROAD, NORTHAMPTON   
5                                          UNIT 17, MAITLAND ROAD, NEEDHAM MARKET, IPSWICH   
6                                        NORTH MALTING THE MALTINGS, FELAW STREET, IPSWICH   
7                                                           19-21 REGENT STREET, CAMBRIDGE   
8                                 UNIT 3 RIVERSIDE HOUSE, 24 LOWER SOUTHEND ROAD, WICKFORD   
9                                       WORKSHOP JOHNS AUTOS, TEMPERANCE PLACE, DARLINGTON   
10                                                         WEMBLEY GARDENS, CAMBOIS, BLYTH   
11                                        SITE 4/5, COQUET ENTERPRISE PARK, AMBLE, MORPETH   
12                                                 SHOP, 68 EASTBOURNE ROAD, MIDDLESBROUGH   
13                                                            UNIT 3, ALBERT ROAD, RUSHDEN   
14                                                                    3 CARR STREET, BLYTH   
15                                                        LINCOLN ROAD, NETTLEHAM, LINCOLN   
16                                                                    1 CHURCH LANE, FIELD   
17                                                        IN WAREHOUSE, ATLAS ROAD, LONDON   
18                                                          CHARFLEETS ROAD, CANVEY ISLAND   
19                                              CRACKENEDGE LANE, DEWSBURY, WEST YORKSHIRE   
21                                                        SHOP, 66 STATION ROAD, UPMINSTER   
22                                                    IMPERIAL WAY, WATFORD, HERTFORDHSIRE   
23                               TROUGH FIELD SUPPLY, DELGATE BANK, WESTON HILLS, SPALDING   
24                               BRANKLEY HOUSE, PLOT 6, FAR END, BOOTHBY GRAFFOE, LINCOLN   
25                                      UNIT 7 CLIFFSIDE TRADE PARK, MOTHERWELL WAY, GRAYS   
26                      SOUTHWICK GOLD GLOVES 1ST FLR, 24 THE GREEN, SOUTHWICK, SUNDERLAND   
27                                           5 WOODSIDE PARADE, 257 WOODSIDE, LEIGH ON SEA   
28                                                            185 FELIXSTOWE ROAD, IPSWICH   
29                                                              2 CORNWALLIS ROAD, LINCOLN   
30                   RIVERS EDGE HOTEL RIVERSIDE PARK, GREEN LANE, FRIARS GOOSE, GATESHEAD   
31                            METER FOR UNIT 7 & UNIT 8, SOUTHCHURCH ROAD, SOUTHEND ON SEA   
32                                   1A MIDDLESBROUGH ROAD WEST, SOUTH BANK, MIDDLESBROUGH   
33                                 N11, MABLETHORPE CHALET PARK, LINKS AVENUE, MABLETHORPE   
34                                                                1-3 DEWEY ROAD, DAGENHAM   
35                                                     UNIT 5, BRINELL WAY, GREAT YARMOUTH   
36                               UNIT 7, TELFORD PARK, TELFORD ROAD, CLACTON ON SEA, ESSEX   
37                                                                 2 HAMPDEN ROAD, HITCHIN   
38                                 UNIT 5D INDUSTRIAL ESTATE, GALLAMORE LANE, MARKET RASEN   
39                                              9/10 2ND METER, DEFENDER COURT, SUNDERLAND   
40                                            15-17 SHOP, WEST PERCY STREET, NORTH SHIELDS   
41                                                                 HAIGH STREET, LIVERPOOL   
42                               FACTORY INSHORE FISH & FROZEN FDS, TOD POINT ROAD, REDCAR   
43                               GLASSTHORPE LANE, HILL FARM, TROUGH, HARPOLE, NORTHAMPTON   
44                                                         BLAKE AVENUE, BARKING, DAGENHAM   
45                           

In [23]:
df.loc[df['Street_In_Postcode'] == 'No']

Single_Line_Address  \
0                                   UNIT 3, 39-41 MOORBRIDGE ROAD, MAIDENHEAD, BERKSHIRE   
4                          METER R/O UNIT 54 UNKNOWN SUPPLY, DALTON PARK, MURTON, SEAHAM   
20                                              TOILETS & PREMISES, PARK ROAD, ASHINGTON   
50                                              DUNE, EAST FARM HOUSE, EMBLETON, ALNWICK   
69                                                      11B MIDDLETON GRANGE, HARTLEPOOL   
75                                                RUSTON HOUSE, WATERSIDE SOUTH, LINCOLN   
76                                     HUNTERS ROAD, SPITAL TONGUES, NEWCASTLE UPON TYNE   
77                                           4/7 UNIT BT B, LYNMOUTH ROAD, NORTH SHIELDS   
78                               MIDWEST BUSINESS PARK, WASHWAY ROAD, HOLBEACH, SPALDING   
79                                                       THE OLD CHAPEL, HEXTON, HITCHIN   
80                                                DOCKING FARM - MEADOW, OULTON, NORWICH   
81                                       UNIT 19, ALLIANCE INDUSTRIAL ESTATE, DARLINGTON   
82                                 20A 1ST & 2ND FLOORS, FREDERICK STREET, SOUTH SHIELDS   
83                                ASHURST, SOUTHGATE PARK, ORTON SOUTHGATE, PETERBOROUGH   
84                                                            8 THE ARCADE, PETERBOROUGH   
85                                                 CAR PARK, CAMPBELL PARK ROAD, HEBBURN   
86                                                  FORNHAM ALL SAINTS, BURY ST. EDMUNDS   
87                                     FRESH WHARF DEVELOPMENT, HIGHBRIDGE ROAD, BARKING   
88                                                    SHOP H, PRINCES STREET, HUNTINGDON   
89                                                                  ORFORD YARD, NORWICH   
90                                                                   17 NEWLANDS, WITHAM   
91            BROOMFIELD GRANGE NURSING HOME, PUDDINGS WOOD LANE, BROOMFIELD, CHELMSFORD   
92                                                        LAND ADJACENT TO HOLLY COTTAGE   
93                 LEMINGTON TRAVELLERS CARAVAN SITE, NEWBURN HAUGH, NEWCASTLE UPON TYNE   
94                                                          SITE OFFICE, STATION, LONDON   
95                                   LAND OFF PLOT 8, DRAIN BANK NORTH, COWBIT, SPALDING   
96                                   UNITS 8A/8B, UNIT 2, RIVERSIDE RETAIL PARK, NORWICH   
97                                                         BINSTORE, BRUFF ROAD, IPSWICH   
98                                             THE PHARMACY, RAVENSWORTH ROAD, GATESHEAD   
99  OFFICE 13 GND FLR, GROUNDSTAR HSE, FREIGHT VILLAGE, WOOLSINGTON, NEWCASTLE UPON TYNE   

    Postcode  \
0    SL6 8LT   
4    SR7 9HU   
20  NE63 9NF   
50  NE66 3XB   
69  TS24 7RS   
75   LN5 7FD   
76   NE2 4NE   
77  NE29 7TY   
78  PE12 8JB   
79   SG5 3HX   
80  NR11 6QZ   
81   DL1 2PA   
82  NE33 5AE   
83   PE2 6YS   
84   PE1 1PY   
85  NE31 2SN   
86  IP28 6JH   
87  IG11 7BP   
88  PE29 3LN   
89   NR1 3TB   
90   CM8 2AP   
91   CM1 7FG   
92   IP8 3JD   
93  NE15 8SG   
94  SE13 5FT   
95  PE12 6FR   
96   NR1 1WR   
97   IP2 8EF   
98  NE11 9AZ   
99  NE13 8BH   

                                                                Expected_Street  \
0                                                                            []   
4                                                                         [nan]   
20                                                          [nan, ARIEL STREET]   
50                                                                        [nan]   
69                                                                        [nan]   
75                                                              [PELHAM STREET]   
76                                                                           []   
77  [BARMOUTH ROAD, DARTMOUTH ROAD, PLYMOUTH ROAD, EXMO

In [24]:
df['Street_In_Postcode'].value_counts()

Yes    70
No     30
Name: Street_In_Postcode, dtype: int64

In [25]:
df['Street_In_Postcode'].value_counts().plot.bar()

<AxesSubplot:>

In [26]:
# check for potential typos (levenshtein accounts for string length)
df['lev_dist'] = df.apply(lambda x: levenshtein.distance(
    x['Single_Line_Address'], x['Expected_Street']), axis=1)
df

Single_Line_Address  \
0                                     UNIT 3, 39-41 MOORBRIDGE ROAD, MAIDENHEAD, BERKSHIRE   
1                                                       4 WORKSHOP, PERCY ROAD, DARLINGTON   
2                        SAXON HOUSE, HENSON WAY, TELFORD WAY INDUSTRIAL ESTATE, KETTERING   
3                                                               1 QUARRY ROAD, NORTHAMPTON   
4                            METER R/O UNIT 54 UNKNOWN SUPPLY, DALTON PARK, MURTON, SEAHAM   
5                                          UNIT 17, MAITLAND ROAD, NEEDHAM MARKET, IPSWICH   
6                                        NORTH MALTING THE MALTINGS, FELAW STREET, IPSWICH   
7                                                           19-21 REGENT STREET, CAMBRIDGE   
8                                 UNIT 3 RIVERSIDE HOUSE, 24 LOWER SOUTHEND ROAD, WICKFORD   
9                                       WORKSHOP JOHNS AUTOS, TEMPERANCE PLACE, DARLINGTON   
10                                                         WEMBLEY GARDENS, CAMBOIS, BLYTH   
11                                        SITE 4/5, COQUET ENTERPRISE PARK, AMBLE, MORPETH   
12                                                 SHOP, 68 EASTBOURNE ROAD, MIDDLESBROUGH   
13                                                            UNIT 3, ALBERT ROAD, RUSHDEN   
14                                                                    3 CARR STREET, BLYTH   
15                                                        LINCOLN ROAD, NETTLEHAM, LINCOLN   
16                                                                    1 CHURCH LANE, FIELD   
17                                                        IN WAREHOUSE, ATLAS ROAD, LONDON   
18                                                          CHARFLEETS ROAD, CANVEY ISLAND   
19                                              CRACKENEDGE LANE, DEWSBURY, WEST YORKSHIRE   
20                                                TOILETS & PREMISES, PARK ROAD, ASHINGTON   
21                                                        SHOP, 66 STATION ROAD, UPMINSTER   
22                                                    IMPERIAL WAY, WATFORD, HERTFORDHSIRE   
23                               TROUGH FIELD SUPPLY, DELGATE BANK, WESTON HILLS, SPALDING   
24                               BRANKLEY HOUSE, PLOT 6, FAR END, BOOTHBY GRAFFOE, LINCOLN   
25                                      UNIT 7 CLIFFSIDE TRADE PARK, MOTHERWELL WAY, GRAYS   
26                      SOUTHWICK GOLD GLOVES 1ST FLR, 24 THE GREEN, SOUTHWICK, SUNDERLAND   
27                                           5 WOODSIDE PARADE, 257 WOODSIDE, LEIGH ON SEA   
28                                                            185 FELIXSTOWE ROAD, IPSWICH   
29                                                              2 CORNWALLIS ROAD, LINCOLN   
30                   RIVERS EDGE HOTEL RIVERSIDE PARK, GREEN LANE, FRIARS GOOSE, GATESHEAD   
31                            METER FOR UNIT 7 & UNIT 8, SOUTHCHURCH ROAD, SOUTHEND ON SEA   
32                                   1A MIDDLESBROUGH ROAD WEST, SOUTH BANK, MIDDLESBROUGH   
33                                 N11, MABLETHORPE CHALET PARK, LINKS AVENUE, MABLETHORPE   
34                                                                1-3 DEWEY ROAD, DAGENHAM   
35                                                     UNIT 5, BRINELL WAY, GREAT YARMOUTH   
36                               UNIT 7, TELFORD PARK, TELFORD ROAD, CLACTON ON SEA, ESSEX   
37                                                                 2 HAMPDEN ROAD, HITCHIN   
38                                 UNIT 5D INDUSTRIAL ESTATE, GALLAMORE LANE, MARKET RASEN   
39                                              9/10 2ND METER, DEFENDER COURT, SUNDERLAND   
40                                            15-17 SHOP, WEST PERCY STREET, NORTH SHIELDS   
41                                                                 HAIGH STREET, LIVERPOOL   
42                           

In [27]:
df1 = df[["Single_Line_Address", "Postcode", "Street_In_Postcode"]]

In [29]:
df1.to_csv("C:/Users/Admin/Identeqchallenge/example_output_data.csv", sep=',', encoding='utf-8', index=False)